In [1]:
import Cython

In [2]:
%load_ext cython

In [3]:
%%cython -a -f
# distutils: language = c++
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

import numpy as np
cimport numpy as np

np.import_array()

from CyRK cimport PreEvalFunc, cysolve_ivp, CySolveOutput
cdef void diffeq(double* dy, double t, double* y, const void* args, PreEvalFunc NA) noexcept nogil:
    # Real dy
    dy[0] = 3.1 * t - y[1]
    dy[1] = y[0] * (0.3 * t * y[1])
    # Extra
    dy[2] = 0.25
    dy[3] = t / 2.

cdef CySolveOutput run():
    
    cdef double[2] t_span = [0.0, 10.0]
    cdef double* t_span_ptr = &t_span[0]
    
    cdef double[2] y0 = [5., 2.]
    cdef double* y0_ptr = &y0[0]
    cdef double[5] t_eval = [0.3, 4.0, 8., 9.10, 9.9]
    cdef double* t_eval_ptr = &t_eval[0]
    cdef int num_t_eval = 5
    
    cdef CySolveOutput cyresult = \
        cysolve_ivp(
            diffeq,
            t_span_ptr,
            y0_ptr,
            2,
            1,
            1.0e-3,
            1.0e-6,
            NULL,
            2, # num extra
            0,
            2000,
            True,
            t_eval_ptr,
            num_t_eval,
            NULL,
            NULL,
            NULL,
            10000.,
            0.0,
            0)

    return cyresult

cdef CySolveOutput res = run()
print('')
print('Integration Done!')


cdef double[4] y_arr = [-999, -999, -999, -999]
cdef double* y_arr_ptr = &y_arr[0]

cdef int i
for i in range(5):
    print(f"teval i = {i} ys= {res.get().solution[4*i]}\t|\t{res.get().solution[4*i+1]}\t|\t{res.get().solution[4*i+2]}\t|\t{res.get().solution[4*i+3]}")

cdef double rr
for r in [0.3, 4.0, 8., 9.10, 9.9]:
    rr = r
    res.get().call(rr, y_arr_ptr)
    print(f"INTERP AT r={rr}: {y_arr_ptr[0]:0.3f}\t|\t{y_arr_ptr[1]:0.3f}\t|\t{y_arr_ptr[2]:0.3f}\t|\t{y_arr_ptr[3]:0.3f}")


Content of stdout:
_cython_magic_8ec2014aa0d4675425dd699e02c0b83159c42fcf.cpp
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\CyRK\cy\dense.cpp(144): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\CyRK\cy\cysolution.cpp(53): warning C5051: attribute [[likely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\CyRK\cy\cysolution.cpp(68): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\CyRK\cy\cysolution.cpp(98): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\cyrk11py311

In [ ]:
# 0.11.1 result:
Finished generating code
Integration Done!
INTERP AT r=0.3: 4.526	|	2.131	|	-999.000	|	-999.000
INTERP AT r=4.0: 1.872	|	21.054	|	-999.000	|	-999.000
INTERP AT r=8.22: 2.500	|	23.547	|	-999.000	|	-999.000
INTERP AT r=9.1: 1.579	|	48.552	|	-999.000	|	-999.000

# v0.11.2a1:
Integration Done!
teval i = 0 ys= 4.526273291353615	|	2.130934826873202	|	0.25	|	0.15
teval i = 1 ys= 1.8716084470161582	|	21.05384255519958	|	0.25	|	2.0
teval i = 2 ys= 0.13355624397337607	|	10.312017312712717	|	0.25	|	4.0
teval i = 3 ys= 1.578715913166198	|	48.55169176359066	|	0.25	|	4.55
teval i = 4 ys= -1.015908770745836	|	57.06848507859018	|	0.25	|	4.95
INTERP AT r=0.3: 4.526	|	2.131	|	0.250	|	0.150
INTERP AT r=4.0: 1.872	|	21.054	|	0.250	|	2.000
INTERP AT r=8.0: 0.134	|	10.312	|	0.250	|	4.000
INTERP AT r=9.1: 1.579	|	48.552	|	0.250	|	4.550
INTERP AT r=9.9: -1.016	|	57.068	|	0.250	|	4.950

In [6]:
import numpy as np
from CyRK import pysolve_ivp

def diffeq(dy, t, y,):
    # Real dy
    dy[0] = 3.1 * t - y[1]
    dy[1] = y[0] * (0.3 * t * y[1])
    # Extra
    dy[2] = 0.25
    dy[3] = t / 2.

t_span = (0.0, 10.0)
y0 = np.asarray([5., 2.], dtype=np.float64)

result = pysolve_ivp(
    diffeq,
    t_span,
    y0,
    method = 'RK45',
    t_eval = None,
    dense_output = True,
    args = None,
    expected_size = 0,
    num_extra = 2,
    first_step = 0.0,
    max_step = 100_000,
    rtol = 1.0e-3,
    atol = 1.0e-6,
    max_num_steps = 0,
    max_ram_MB = 2000,
    pass_dy_as_arg = True
    )

print(result.y)

print(result(0.3))

print(result(np.asarray([0.3, 4.0, 8., 9.10, 9.9])))

[[ 5.00000000e+00  4.81923129e+00  3.97734019e+00  3.16621612e+00
   6.47132288e-01 -1.46290211e+00 -2.05651329e+00 -8.98427679e-01
   1.23410081e+00  2.70476777e+00  1.97436005e+00 -4.05484336e-01
  -2.13157571e+00 -2.12317883e+00 -7.30858100e-01  1.62814014e+00
   2.59651151e+00  1.58438102e+00 -9.91304342e-01 -2.34264539e+00
  -1.84686556e+00 -1.81486448e-01  2.18604373e+00  2.48215975e+00
   8.78524981e-01 -1.23711911e+00 -2.32923624e+00 -1.78714363e+00
  -1.46045369e-01  2.18744114e+00  2.43319389e+00  7.59796956e-01
  -1.34272165e+00 -2.36110709e+00 -1.85839711e+00 -2.45599900e-01
   2.10029201e+00  2.44007512e+00  7.66362395e-01 -1.33297063e+00
  -2.36230734e+00 -1.87918536e+00 -2.82910371e-01  2.06103482e+00
   2.43449671e+00  7.53642725e-01 -1.34189190e+00 -2.36624103e+00
  -1.88201967e+00 -2.91727436e-01  2.04518502e+00  2.42515288e+00
   7.36020153e-01 -1.35557261e+00 -2.37010914e+00 -2.42267219e+00]
 [ 2.00000000e+00  2.01396550e+00  4.02489894e+00  7.89885437e+00
   1.3156

In [ ]:
INTERP AT r=0.3: 4.526	|	2.131	|	0.250	|	0.150
INTERP AT r=4.0: 1.872	|	21.054	|	0.250	|	2.000
INTERP AT r=8.0: 0.134	|	10.312	|	0.250	|	4.000
INTERP AT r=9.1: 1.579	|	48.552	|	0.250	|	4.550
INTERP AT r=9.9: -1.016	|	57.068	|	0.250	|	4.950